# Naive bayes 模型

#### 通俗的说，判断某类别出现的概率，就是该类别出现的概率，以及数据在该类别下此pattern出现的概率的乘积。
#### 举例来说就是：你看到了外面1.云很厚 2.天很暗 3. 气压很低，那么想问一下，今天会下雨的概率是多少？
#### 其实今天下雨的概率就是这个地方一年下雨的概率(沙漠和热带雨林这个概率不同)*如果下雨那么天上的云很厚的概率*如果下雨那么天很暗的概率*如果下雨那么气压很低的概率。

# 1.导入数据

In [1]:
import numpy as np
import pandas as pd

# 2. 创建数据

In [2]:
def createData():
    dataDic = {"age":['young', 'young', 'young', 'young', 'young', 'mid', 'mid', 'mid', 'mid', 'mid', 'old', 'old', 'old', 'old', 'old'], 
               "work":['no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no'], 
               "house":['no', 'no', 'no', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no'], 
               "credit":['normal', 'good', 'good', 'normal', 'normal', 'normal', 'good', 'good', 'verygood', 'verygood', 'verygood', 'good', 'good', 'verygood', 'normal'], 
               "label":['no', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']}
    df = pd.DataFrame(dataDic,columns=['age','work','house','credit','label'])
    return df
createData()

,age,work,house,credit,label
0,young,no,no,normal,no
1,young,no,no,good,no
2,young,yes,no,good,yes
3,young,yes,yes,normal,yes
4,young,no,no,normal,no
5,mid,no,no,normal,no
6,mid,no,no,good,no
7,mid,yes,yes,good,yes
8,mid,no,yes,verygood,yes
9,mid,no,yes,verygood,yes


# 3. 朴素贝叶斯模型

#### 朴素体现在假设比较理想：所有变量之间是独立的

In [3]:
class NB:
    def __init__(self, data):
        self.data = data
    def labelFrequency(self, data): # 统计不同类别的标签对应的概率
        labelDic = {}
        valueCount = data.label.value_counts()
        valueFreq = valueCount*1.0/sum(valueCount)
        for i in valueFreq.index:
            labelDic[i] = valueFreq[i]
        return labelDic
    def yesNoDataFrequency(self, data, label):
        dataFrequencyDict = {}
        dataSub = data.loc[data.label==label,:]
        for i in range(dataSub.shape[1]-1):
            featureCount = dataSub.iloc[:,i].value_counts()
            featureFreq = featureCount*1.0/sum(featureCount)
            feat = dataSub.columns[i]
            for j in featureFreq.index:
                featName = '@'.join([feat,j]) # 类似于one_hot编码
                dataFrequencyDict[featName] = featureFreq[j]
        return dataFrequencyDict
    def calProb(self, data, label, unknown): # unknown 应该是个list类型
        labelFreqDic = self.labelFrequency(data)
        dataFreqDic = self.yesNoDataFrequency(data, label)
        prob = labelFreqDic[label]
        for i,v in enumerate(unknown):
            tmpName = '@'.join([data.columns[i], v])
            prob *= dataFreqDic[tmpName]
        return prob
    def predictLabel(self, unknown):
        maxLabel = ''
        maxValue = -1
        labels = self.data.iloc[:,-1].unique()
        probDic = {}
        for i in labels:
            probTmp = self.calProb(self.data, i, unknown)
            probDic[i] = probTmp
        for k,v in probDic.iteritems():
            if (maxValue<v):
                maxValue = v
                maxLabel = k
        return maxLabel

In [4]:
data = createData()
NBModel = NB(data)
unknown = ['young','no','no','normal']
predictLabel = NBModel.predictLabel(unknown)
print "预测数据: ['young','no','no','normal']"
print "预测结果: %s"%predictLabel

预测数据: ['young','no','no','normal']
预测结果: no
